# Imports

In [2]:
from ranx import Qrels, Run, evaluate, compare

In [14]:
#? compare
#? evaluate

In [4]:
import numpy as np
import os.path as osp
import pickle, json, random

In [5]:
def json_load(path):
    with open(path, 'r') as fid:
        data_ = json.load(fid)
    return data_

In [7]:
dataset_name = 'viquae_for_rrt'
delg_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python', 'delg')
data_dir = osp.join(delg_dir, dataset_name)

In [8]:
!ls $delg_dir/dev_run_dict_*

/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_100
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_200
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_300
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_400
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_500
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_600
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_700
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_800
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/dev_run_dict_900


In [33]:
qrels = Qrels.from_file(osp.join(delg_dir, 'tuto_qrels_dict'), kind="json")

In [34]:
max_sequence_len = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
max_sequence_len = [100, 200, 300, 400, 500, 600, 700, 800, 900]
max_sequence_len = [500, 800]


In [35]:
runs = []
for length in max_sequence_len:
    filename = osp.join(delg_dir, 'tuto_run_dict_'+str(length))
    print(filename)
    run_dict = json_load(filename)
    runs.append(Run(run_dict))

/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/tuto_run_dict_500
/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/tuto_run_dict_800


In [36]:
metrics = ["map", "mrr", "precision", "hit_rate", "recall"]
kappas  = [1, 5, 10]
m_list  = [metric for metric in metrics]

for i in range(len(kappas)):
    m_list.extend([metric+'@'+str(kappas[i]) for metric in metrics])

metrics = [element for element in m_list if element not in ['precision',  'hit_rate',  'recall',  'map@1',  'mrr@1', 'hit_rate@1', 'map@5',  'mrr@5',  'map@10',  'mrr@10']]

In [37]:
qrels.save('zut_qrels')

In [31]:
ls

data/
delg_scripts/
evaluate_revisited.ipynb
evaluate_revisited.py
evaluate_viquae.ipynb
evaluate_viquae.py
experiment_new_viquae-Copy1.ipynb
experiment_new_viquae.ipynb
experiment.py
experiment_viquae_fast.py
experiment_viquae.ipynb
experiment_viquae.py
experiment_viquae_transfer_learning-Copy1.ipynb
experiment_viquae_transfer_learning.ipynb
experiment_viquae_transfer_learning_train-Copy1.ipynb
experiment_viquae_transfer_learning_train.ipynb
foo.py
history.py
logs/
models/
prepare_topk_revisited.ipynb
prepare_topk_viquae.ipynb
prepare_topk_viquae_test.ipynb
prepare_topk_viquae_train.ipynb
ranx_compare_ablation_study.ipynb
README.md
rrt_gld_ckpts/
rrt_scripts/
swagg.py
swing.py
to_bin_experiment_new_viquae.ipynb
to_bin_experiment_viquae.ipynb
to_bin_gnd_exploration.ipynb
tools/
tutoqrels_dict
tutorun_dict_500
Untitled.ipynb
untitled.txt
utils/
viquae_for_rrt_training_data_preparation.ipynb
zut_qrels


In [29]:
evaluate(qrels, runs[1], m_list)

{'map': 0.30367103352657826,
 'mrr': 0.4656070796279204,
 'precision': 0.13010005331583033,
 'hit_rate': 0.84,
 'recall': 0.8353333333333333,
 'map@1': 0.12387409396936318,
 'mrr@1': 0.3848,
 'precision@1': 0.3848,
 'hit_rate@1': 0.3848,
 'recall@1': 0.12387409396936318,
 'map@5': 0.17727036704097332,
 'mrr@5': 0.4441866666666667,
 'precision@5': 0.19888,
 'hit_rate@5': 0.536,
 'recall@5': 0.21326514242793831,
 'map@10': 0.20285686253505622,
 'mrr@10': 0.45554571428571433,
 'precision@10': 0.16256,
 'hit_rate@10': 0.6208,
 'recall@10': 0.2925221308673156}

In [17]:
# Compares different runs and performs statistical tests (Fisher's Randomization test)
report = compare(
    qrels,
    runs,
    metrics=metrics,
    max_p=0.01,  # P-value threshold
    # Use `fisher` for Fisher's Randomization Test (slower) 
    # or `student` for Two-sided Paired Student's t-Test (faster)
    rounding_digits=3,
    show_percentages=True,
)

In [18]:
report

#    Model      MAP    MRR    P@1    Recall@1    P@5    Hit_Rate@5    Recall@5    P@10    Hit_Rate@10    Recall@10
---  -------  -----  -----  -----  ----------  -----  ------------  ----------  ------  -------------  -----------
a    run_1     30.4   46.6   38.5        12.4   19.9          53.6        21.3    16.3           62.1         29.3
b    run_2     30.4   46.6   38.5        12.4   19.9          53.6        21.3    16.3           62.1         29.3
c    run_3     30.4   46.6   38.5        12.4   19.9          53.6        21.3    16.3           62.1         29.3
d    run_4     30.4   46.6   38.5        12.4   19.9          53.6        21.3    16.3           62.1         29.3
e    run_5     30.4   46.6   38.5        12.4   19.9          53.6        21.3    16.3           62.1         29.3
f    run_6     30.4   46.6   38.5        12.4   19.9          53.6        21.3    16.3           62.1         29.3
g    run_7     30.4   46.6   38.5        12.4   19.9          53.6        21.3  

# Default Parameters

In [30]:
set_name = 'dev'
set_name = 'tuto'
eval_set_name = set_name
train_txt = set_name+'_query.txt'
test_txt = (set_name+'_query.txt', set_name+'_selection.txt')
train_data_dir = 'data/viquae_for_rrt'
test_data_dir  = 'data/viquae_for_rrt'
test_gnd_file = 'gnd_'+ set_name+'.pkl'
train_gnd_file = 'gnd_'+ set_name+'.pkl'
desc_name = 'r50_gldv2'
split_char  = ';;'
sampler = 'random'

In [31]:
cpu = False
cudnn_flag = 'benchmark'
temp_dir = osp.join('logs', 'temp')
resume = '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt'
seed = 0

In [32]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [38]:
train_data_dir = '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt'

In [42]:
batch_size      = 16
test_batch_size = 16
max_sequence_len = 1000
sampler = 'random'

In [50]:
name = 'rrt'
num_global_features = 2048  
num_local_features = 128  
seq_len = 1004
dim_K = 256
dim_feedforward = 1024
nhead = 4
num_encoder_layers = 6
dropout = 0.0 
activation = "relu"
normalize_before = False

In [44]:
num_workers = 8  # number of workers used ot load the data
pin_memory  = True  # use the pin_memory option of DataLoader 
num_candidates = 100
recalls = [1, 5, 10]

In [76]:
ks = recalls

In [33]:
device = torch.device('cuda:0' if torch.cuda.is_available() and not cpu else 'cpu')
torch.manual_seed(seed)

In [34]:
if cudnn_flag == 'deterministic':
    setattr(cudnn, cudnn_flag, True)

# Utilis Functions

In [35]:
def read_file(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

In [36]:
def get_sets(desc_name, 
        train_data_dir, test_data_dir, train_txt, 
        test_txt, train_gnd_file,  test_gnd_file,
        max_sequence_len, split_char):
    ####################################################################################################################################
    if len(train_txt) == 2:
        train_gnd_data  = None if train_gnd_file is None else pickle_load(osp.join(train_data_dir, train_gnd_file))
        train_lines     = read_file(osp.join(train_data_dir, train_txt[1]))
        train_q_lines   = read_file(osp.join(train_data_dir, train_txt[0]))
        train_samples   = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in train_lines]
        train_q_samples = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in train_q_lines]
        train_set       = FeatureDataset(train_data_dir, train_samples,   desc_name, max_sequence_len, gnd_data=train_gnd_data)
        query_train_set = FeatureDataset(train_data_dir, train_q_samples, desc_name, max_sequence_len, gnd_data=train_gnd_data)
    else:
        train_gnd_data  = None if train_gnd_file is None else pickle_load(osp.join(train_data_dir, train_gnd_file))
        train_lines     = read_file(osp.join(train_data_dir, train_txt))
        train_samples   = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in train_lines]
        train_set       = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len, gnd_data=train_gnd_data)
        query_train_set = FeatureDataset(train_data_dir, train_samples, desc_name, max_sequence_len, gnd_data=train_gnd_data)
        ####################################################################################################################################
    test_gnd_data   = None if test_gnd_file is None else pickle_load(osp.join(test_data_dir, test_gnd_file))
    query_lines     = read_file(osp.join(test_data_dir, test_txt[0]))
    gallery_lines   = read_file(osp.join(test_data_dir, test_txt[1]))
    query_samples   = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in query_lines]
    gallery_samples = [(line.split(split_char)[0], int(line.split(split_char)[1]), int(line.split(split_char)[2]), int(line.split(split_char)[3])) for line in gallery_lines]
    gallery_set     = FeatureDataset(test_data_dir, gallery_samples, desc_name, max_sequence_len)
    query_set       = FeatureDataset(test_data_dir, query_samples,   desc_name, max_sequence_len, gnd_data=test_gnd_data)
        
    return (train_set, query_train_set), (query_set, gallery_set)



In [37]:
class MetricLoaders(NamedTuple):
    train: DataLoader
    num_classes: int
    query: DataLoader
    query_train: DataLoader
    set_name: str = ''
    eval_set_name: str = ''
    gallery: Optional[DataLoader] = None

In [40]:
def get_loaders(desc_name, train_data_dir, 
    batch_size, test_batch_size, 
    num_workers, pin_memory, 
    sampler, recalls, set_name, 
    eval_set_name, train_gnd_file,
    num_candidates=100,):

    (train_set, query_train_set), (query_set, gallery_set) = get_sets(desc_name, 
        train_data_dir=train_data_dir,
        test_data_dir=train_data_dir,
        train_txt=set_name+'_query.txt',
        test_txt=(set_name+'_query.txt', set_name+'_selection.txt'),
        test_gnd_file=test_gnd_file, 
        train_gnd_file=train_gnd_file,
        split_char=split_char,
        max_sequence_len=500)

    if sampler == 'random':
        train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)
    elif sampler == 'triplet':
        train_nn_inds = osp.join(train_data_dir, set_name + '_nn_inds_%s.pkl'%desc_name)
        gnd_data = train_set.gnd_data['gnd']
        train_sampler = TripletSampler(query_train_set.targets, batch_size, train_nn_inds, num_candidates, gnd_data)
    else:
        raise ValueError('Invalid choice of sampler ({}).'.format(sampler))
    train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
    query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
        
    query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
    gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

    return MetricLoaders(train=train_loader, query_train=query_train_loader, query=query_loader, gallery=gallery_loader, num_classes=len(train_set.categories),set_name=set_name,eval_set_name=eval_set_name), recalls


In [41]:
def mean_average_precision_viquae_rerank(
    model: nn.Module,
    cache_nn_inds: torch.Tensor,
    query_global: torch.Tensor, query_local: torch.Tensor, query_mask: torch.Tensor, query_scales: torch.Tensor, query_positions: torch.Tensor,
    gallery_global: torch.Tensor, gallery_local: torch.Tensor, gallery_mask: torch.Tensor, gallery_scales: torch.Tensor, gallery_positions: torch.Tensor,
    ks: List[int],
    gnd) -> Dict[str, float]:

    device = next(model.parameters()).device
    query_global    = query_global.to(device)
    query_local     = query_local.to(device)
    query_mask      = query_mask.to(device)
    query_scales    = query_scales.to(device)
    query_positions = query_positions.to(device)

    num_samples, top_k = cache_nn_inds.size()
    top_k = min(100, top_k)
    
    sizes = [len(gnd['simlist'][i]) for i in range(len(gnd['simlist']))]
    
    gallery_global    = fill_in_and_pad(gallery_global, query_global, sizes)
    gallery_local     = fill_in_and_pad(gallery_local, query_local, sizes)
    gallery_mask      = fill_in_and_pad(gallery_mask, query_mask, sizes)
    gallery_scales    = fill_in_and_pad(gallery_scales, query_scales, sizes)
    gallery_positions = fill_in_and_pad(gallery_positions, query_positions, sizes)


    ########################################################################################
    ## Evaluation
    eval_nn_inds = deepcopy(cache_nn_inds.cpu().data.numpy())

    ## Exclude the junk images as in DELG (https://github.com/tensorflow/models/blob/44cad43aadff9dd12b00d4526830f7ea0796c047/research/delf/delf/python/detect_to_retrieve/image_reranking.py#L190)
    #for i in range(num_samples):
    #    junk_ids = gnd['gnd'][i]['r_neg']
    #    all_ids = eval_nn_inds[i]
    #    pos = np.in1d(all_ids, junk_ids)
    #    neg = np.array([not x for x in pos])
    #    new_ids = np.concatenate([np.arange(len(all_ids))[neg], np.arange(len(all_ids))[pos]])
    #    new_ids = all_ids[new_ids]
    #    eval_nn_inds[i] = new_ids
    eval_nn_inds = torch.from_numpy(eval_nn_inds)
    
    scores = []
    for i in tqdm(range(top_k)):
        nnids = eval_nn_inds[:, i]
        topk_scores =  []
        for iterator in range(nnids.size(dim=0)):
            index_global = gallery_global[iterator, nnids[iterator]]
            index_local = gallery_local[iterator, nnids[iterator]]
            index_mask = gallery_mask[iterator, nnids[iterator]]
            index_scales = gallery_scales[iterator, nnids[iterator]]
            index_positions = gallery_positions[iterator, nnids[iterator]]
            
            index_global = index_global.unsqueeze(dim=0)
            index_global = index_global.type(torch.float32)

            index_local = index_local.unsqueeze(dim=0)
            index_local = index_local.type(torch.float32)

            index_mask = index_mask.unsqueeze(dim=0)
            index_mask = index_mask.type(torch.bool)

            index_scales = index_scales.unsqueeze(dim=0)
            index_scales = index_scales.type(torch.int64)

            index_positions = index_positions.unsqueeze(dim=0)
            index_positions = index_positions.type(torch.float32)
            
            q_global = query_global[iterator].unsqueeze(dim=0)
            q_local = query_local[iterator].unsqueeze(dim=0)
            q_mask = query_mask[iterator].unsqueeze(dim=0)
            q_scales = query_scales[iterator].unsqueeze(dim=0)
            q_positions = query_positions[iterator].unsqueeze(dim=0)

            iter_scores = model(
            q_global, q_local, q_mask, q_scales, q_positions,
                index_global.to(device),
                index_local.to(device),
                index_mask.to(device),
                index_scales.to(device),
                index_positions.to(device))
            
            topk_scores.append(iter_scores.cpu().data)
        
        current_scores = torch.from_numpy(np.stack(topk_scores, axis=0)).squeeze(1)
        torch.cuda.empty_cache()        
        scores.append(current_scores.cpu().data)
    
    
    scores = torch.stack(scores, -1) # nb_queries x 100
    closest_dists, indices = torch.sort(scores, dim=-1, descending=True)
    closest_indices = torch.gather(eval_nn_inds, -1, indices)
    ranks = deepcopy(eval_nn_inds)
    ranks[:, :top_k] = deepcopy(closest_indices)
    ranks = ranks.cpu().data.numpy()
    ranks = remove_padded_indices(ranks, eval_nn_inds, sizes)
    
    out = compute_metrics('viquae', ranks.T, gnd['gnd'], sizes, kappas=ks)

    ########################################################################################  
    
    return out

In [49]:
def get_model(num_global_features, num_local_features, seq_len, dim_K, dim_feedforward, nhead, num_encoder_layers, dropout, activation, normalize_before):
    return MatchERT(d_global=num_global_features, d_model=num_local_features, seq_len=seq_len, d_K=dim_K, nhead=nhead, num_encoder_layers=num_encoder_layers, 
            dim_feedforward=dim_feedforward, dropout=dropout, activation=activation, normalize_before=normalize_before)

In [60]:
def evaluate_viquae(
        model: nn.Module,
        cache_nn_inds: torch.Tensor,
        query_loader: DataLoader,
        gallery_loader: DataLoader,
        recall: List[int]):
    model.eval()
    device = next(model.parameters()).device
    to_device = lambda x: x.to(device, non_blocking=True)

    query_global, query_local, query_mask, query_scales, query_positions, query_names = [], [], [], [], [], []
    gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = [], [], [], [], [], []

    with torch.no_grad():
        for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80):
            q_global, q_local, q_mask, q_scales, q_positions, _, q_names = entry
            query_global.append(q_global.cpu())
            query_local.append(q_local.cpu())
            query_mask.append(q_mask.cpu())
            query_scales.append(q_scales.cpu())
            query_positions.append(q_positions.cpu())
            query_names.extend(list(q_names))
            torch.cuda.empty_cache()

        query_global    = torch.cat(query_global, 0)
        query_local     = torch.cat(query_local, 0)
        query_mask      = torch.cat(query_mask, 0)
        query_scales    = torch.cat(query_scales, 0)
        query_positions = torch.cat(query_positions, 0)

        for entry in tqdm(gallery_loader, desc='Extracting gallery features', leave=False, ncols=80):
            g_global, g_local, g_mask, g_scales, g_positions, _, g_names = entry
            gallery_global.append(g_global.cpu())
            gallery_local.append(g_local.cpu())
            gallery_mask.append(g_mask.cpu())
            gallery_scales.append(g_scales.cpu())
            gallery_positions.append(g_positions.cpu())
            gallery_names.extend(list(g_names))
            torch.cuda.empty_cache()
            
        gallery_global    = torch.cat(gallery_global, 0)
        gallery_local     = torch.cat(gallery_local, 0)
        gallery_mask      = torch.cat(gallery_mask, 0)
        gallery_scales    = torch.cat(gallery_scales, 0)
        gallery_positions = torch.cat(gallery_positions, 0)

        torch.cuda.empty_cache()
        evaluate_function = partial(mean_average_precision_viquae_rerank, model=model, cache_nn_inds=cache_nn_inds,
            query_global=query_global, query_local=query_local, query_mask=query_mask, query_scales=query_scales, query_positions=query_positions, 
            gallery_global=gallery_global, gallery_local=gallery_local, gallery_mask=gallery_mask, gallery_scales=gallery_scales, gallery_positions=gallery_positions, 
            ks=recall, 
            gnd=query_loader.dataset.gnd_data,
        )
        metrics = evaluate_function()
    return metrics 

In [70]:
def fill_in_and_pad(gallery_in, query, sizes):
    shape = list(query.shape)
    shape.insert(1, 100)
    gallery_out = torch.zeros(shape)
    size = 0
    counter = 0
    for i in range(gallery_out.size(dim=0)):
        for j in range(gallery_out.size(dim=1)):
            if j < sizes[i]:
                gallery_out[i][j] = gallery_in[counter]
                counter += 1
    
    return gallery_out

In [71]:
def remove_padded_indices(rankings, nn_inds, sizes):
    # rankings.shape -> (nb_queries x 100)
    assert(len(rankings) == len(sizes))
    for i in range(len(sizes)):
        assert(max(nn_inds[i, :sizes[i]]) < sizes[i])
        rankings[i, :sizes[i]] = np.array([value for value in rankings[i] if value < sizes[i]])
        rankings[i, sizes[i]:] = nn_inds[i, sizes[i]:]
        
    return rankings

# Core experiments

In [43]:
if sampler == 'random':
   train_sampler = BatchSampler(RandomSampler(train_set), batch_size=batch_size, drop_last=False)

In [45]:
train_loader = DataLoader(train_set, batch_sampler=train_sampler, num_workers=num_workers, pin_memory=pin_memory)
query_train_loader = DataLoader(query_train_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [46]:
query_loader   = DataLoader(query_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)
gallery_loader = DataLoader(gallery_set, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [47]:
loaders, recall_ks = get_loaders(desc_name=desc_name,
    train_data_dir=train_data_dir, 
    batch_size=36, test_batch_size=36, 
    num_workers=8, pin_memory=True, 
    sampler='random', recalls=[1, 5, 10], 
    set_name=set_name, eval_set_name=None,
    train_gnd_file=None, num_candidates=100)

In [48]:
model_ingredient = Ingredient('model', interactive=True)

In [51]:
model = get_model(num_global_features,num_local_features,seq_len,dim_K,dim_feedforward,nhead,num_encoder_layers,dropout,activation,normalize_before)

In [52]:
if resume is not None:
   checkpoint = torch.load(resume, map_location=torch.device('cpu'))
   model.load_state_dict(checkpoint['state'], strict=True)

In [53]:
model.to(device)
model.eval()

MatchERT(
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Linear(in_features=1024, out_features=128, bias=True)
        (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.0, inplace=False)
        (dropout2): Dropout(p=0.0, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=128, out_features=128, bias=True)
        )
        (linear1): Linear(in_features=128, out_features=1024, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (linear2): Lin

In [84]:
loaders.set_name

'tuto'

In [54]:
loaders.query.dataset.desc_name, loaders.query.dataset.desc_name, loaders.query.dataset.data_dir

('r50_gldv2',
 'r50_gldv2',
 '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt')

In [55]:
loaders.query

In [56]:
nn_inds_path = osp.join(loaders.query.dataset.data_dir, set_name+'_nn_inds_%s.pkl'%loaders.query.dataset.desc_name)
cache_nn_inds = torch.from_numpy(pickle_load(nn_inds_path)).long()

In [57]:
nn_inds_path

'/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt/tuto_nn_inds_r50_gldv2.pkl'

In [58]:
cache_nn_inds.size()

torch.Size([120, 100])

In [59]:
num_samples, top_k = cache_nn_inds.size()
top_k = min(top_k, 100)
top_k

100

In [61]:
model.eval()
device = next(model.parameters()).device
to_device = lambda x: x.to(device, non_blocking=True)

query_global, query_local, query_mask, query_scales, query_positions, query_names = [], [], [], [], [], []
gallery_global, gallery_local, gallery_mask, gallery_scales, gallery_positions, gallery_names = [], [], [], [], [], []

In [62]:
for entry in tqdm(query_loader, desc='Extracting query features', leave=False, ncols=80):
    q_global, q_local, q_mask, q_scales, q_positions, _, q_names = entry
    query_global.append(q_global.cpu())
    query_local.append(q_local.cpu())
    query_mask.append(q_mask.cpu())
    query_scales.append(q_scales.cpu())
    query_positions.append(q_positions.cpu())
    query_names.extend(list(q_names))
    torch.cuda.empty_cache()

In [63]:
query_global    = torch.cat(query_global, 0)
query_local     = torch.cat(query_local, 0)
query_mask      = torch.cat(query_mask, 0)
query_scales    = torch.cat(query_scales, 0)
query_positions = torch.cat(query_positions, 0)

In [64]:
for entry in tqdm(gallery_loader, desc='Extracting gallery features', leave=False, ncols=80):
    g_global, g_local, g_mask, g_scales, g_positions, _, g_names = entry
    gallery_global.append(g_global.cpu())
    gallery_local.append(g_local.cpu())
    gallery_mask.append(g_mask.cpu())
    gallery_scales.append(g_scales.cpu())
    gallery_positions.append(g_positions.cpu())
    gallery_names.extend(list(g_names))
    torch.cuda.empty_cache()

In [65]:
gallery_global    = torch.cat(gallery_global, 0)
gallery_local     = torch.cat(gallery_local, 0)
gallery_mask      = torch.cat(gallery_mask, 0)
gallery_scales    = torch.cat(gallery_scales, 0)
gallery_positions = torch.cat(gallery_positions, 0)

In [66]:
gallery_global.shape, gallery_local.shape, gallery_mask.shape, gallery_scales.shape, gallery_positions.shape

(torch.Size([5858, 2048]),
 torch.Size([5858, 500, 128]),
 torch.Size([5858, 500]),
 torch.Size([5858, 500]),
 torch.Size([5858, 500, 2]))

In [67]:
device = next(model.parameters()).device
query_global    = query_global.to(device)
query_local     = query_local.to(device)
query_mask      = query_mask.to(device)
query_scales    = query_scales.to(device)
query_positions = query_positions.to(device)

num_samples, top_k = cache_nn_inds.size()
top_k = min(100, top_k)

In [68]:
gnd_data = pickle_load(osp.join(data_dir, test_gnd_file))
gnd = pickle_load(osp.join(data_dir, test_gnd_file))

In [69]:
sizes = [len(gnd['simlist'][i]) for i in range(len(gnd['simlist']))]

In [72]:
gallery_global    = fill_in_and_pad(gallery_global, query_global, sizes)
gallery_local     = fill_in_and_pad(gallery_local, query_local, sizes)
gallery_mask      = fill_in_and_pad(gallery_mask, query_mask, sizes)
gallery_scales    = fill_in_and_pad(gallery_scales, query_scales, sizes)
gallery_positions = fill_in_and_pad(gallery_positions, query_positions, sizes)

In [73]:
eval_nn_inds = deepcopy(cache_nn_inds.cpu().data.numpy())

In [74]:
## Exclude the junk images as in DELG (https://github.com/tensorflow/models/blob/44cad43aadff9dd12b00d4526830f7ea0796c047/research/delf/delf/python/detect_to_retrieve/image_reranking.py#L190)
#for i in range(num_samples):
#    junk_ids = gnd['gnd'][i]['r_neg']
#    all_ids = eval_nn_inds[i]
#    pos = np.in1d(all_ids, junk_ids)
#    neg = np.array([not x for x in pos])
#    new_ids = np.concatenate([np.arange(len(all_ids))[neg], np.arange(len(all_ids))[pos]])
#    new_ids = all_ids[new_ids]
#    eval_nn_inds[i] = new_ids
eval_nn_inds = torch.from_numpy(eval_nn_inds)

In [75]:
scores = []
for i in tqdm(range(top_k)):
    nnids = eval_nn_inds[:, i]
    topk_scores =  []
    for iterator in range(nnids.size(dim=0)):
        index_global = gallery_global[iterator, nnids[iterator]]
        index_local = gallery_local[iterator, nnids[iterator]]
        index_mask = gallery_mask[iterator, nnids[iterator]]
        index_scales = gallery_scales[iterator, nnids[iterator]]
        index_positions = gallery_positions[iterator, nnids[iterator]]

        index_global = index_global.unsqueeze(dim=0)
        index_global = index_global.type(torch.float32)

        index_local = index_local.unsqueeze(dim=0)
        index_local = index_local.type(torch.float32)

        index_mask = index_mask.unsqueeze(dim=0)
        index_mask = index_mask.type(torch.bool)

        index_scales = index_scales.unsqueeze(dim=0)
        index_scales = index_scales.type(torch.int64)

        index_positions = index_positions.unsqueeze(dim=0)
        index_positions = index_positions.type(torch.float32)

        q_global = query_global[iterator].unsqueeze(dim=0)
        q_local = query_local[iterator].unsqueeze(dim=0)
        q_mask = query_mask[iterator].unsqueeze(dim=0)
        q_scales = query_scales[iterator].unsqueeze(dim=0)
        q_positions = query_positions[iterator].unsqueeze(dim=0)

        iter_scores = model(
        q_global, q_local, q_mask, q_scales, q_positions,
            index_global.to(device),
            index_local.to(device),
            index_mask.to(device),
            index_scales.to(device),
            index_positions.to(device))

        topk_scores.append(iter_scores.cpu().data)

    current_scores = torch.from_numpy(np.stack(topk_scores, axis=0)).squeeze(1)
    torch.cuda.empty_cache()        
    scores.append(current_scores.cpu().data)


100%|██████████| 100/100 [01:24<00:00,  1.19it/s]


In [91]:
from ranx import Qrels, Run, evaluate
def compute_metrics(dataset, ranks, gnd, sizes=[], kappas=[1, 5, 10], set_name=None, max_sequence_len=None):
    # old evaluation protocol
    if dataset.startswith('classic'):
        map, aps, _, _ = compute_map(ranks, gnd)
        out = {'map': np.around(map*100, decimals=3)}
        print('>> {}: mAP {:.2f}'.format(dataset, out['map']))
     
    # new evaluation protocol for revisited dataset
    elif dataset.startswith('revisited'):
        
        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['easy']])
            g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['hard']])
            gnd_t.append(g)
        mapE, apsE, mprE, prsE = compute_map(ranks, gnd_t, kappas)

        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['easy'], gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk']])
            gnd_t.append(g)
        mapM, apsM, mprM, prsM = compute_map(ranks, gnd_t, kappas)

        gnd_t = []
        for i in range(len(gnd)):
            g = {}
            g['ok'] = np.concatenate([gnd[i]['hard']])
            g['junk'] = np.concatenate([gnd[i]['junk'], gnd[i]['easy']])
            gnd_t.append(g)
        mapH, apsH, mprH, prsH = compute_map(ranks, gnd_t, kappas)


        out = {
            'E_map': np.around(mapE*100, decimals=2),
            'M_map': np.around(mapM*100, decimals=2),
            'H_map': np.around(mapH*100, decimals=2),
            'E_mp':  np.around(mprE*100, decimals=2),
            'M_mp':  np.around(mprM*100, decimals=2),
            'H_mp':  np.around(mprH*100, decimals=2),
            # 'apsE': apsE.tolist(),
            # 'apsM': apsM.tolist(),
            # 'apsH': apsH.tolist(),
        }

        # with open('medium.txt', 'w') as f:
        #     f.write('\n'.join([str(v) for v in apsM]))
        # with open('hard.txt', 'w') as f:
        #     f.write('\n'.join([str(v) for v in apsH]))
        # with open('easy.txt', 'w') as f:
        #     f.write('\n'.join([str(v) for v in apsE]))

        print('>> {}: mAP E: {}, M: {}, H: {}'.format(dataset, out['E_map'], out['M_map'], out['H_map']))
        print('>> {}: mP@k{} E: {}, M: {}, H: {}'.format(dataset, kappas, out['E_mp'], out['M_mp'], out['H_mp']))
    
    # new evaluation protocol for viquae dataset
    elif dataset.startswith('viquae'):
        metrics = ["map", "mrr", "precision", "hit_rate", "recall"]
        m_list = [metric for metric in metrics]
        
        for i in range(len(kappas)):
            m_list.extend([metric+'@'+str(kappas[i]) for metric in metrics])
        
        qrels_dict = {}
        run_dict = {}
        
        for i in range(ranks.T.shape[0]):
            size = sizes[i]
            q_str = "q_"+str(int(i))
            ok_inds = np.concatenate([gnd[i]['r_easy'], gnd[i]['r_hard']])
            
            if len(ok_inds) == 0:
                qrels_dict[q_str] = {"DUMMY_RUN": 0}
            else:
                qrels_dict[q_str] = dict([('d_' + str(int(key)), 1) for key in ok_inds])
            
            run_dict[q_str]   = dict([('d_' + str(int(key)), 1) for key in ranks[:size,i]])
            
        qrels = Qrels(qrels_dict)
        run = Run(run_dict)
        
        if set_name:
            json_save(set_name+'_qrels_dict', qrels_dict)
            json_save(set_name+'_run_dict_'+str(max_sequence_len), run_dict)
        
        out = evaluate(qrels, run, m_list)
        
        for key, value in out.items():
            out[key] = np.around(value*100, decimals=2)
    print(out)
    
    return out

In [78]:
scores = torch.stack(scores, -1) # nb_queries x 100
closest_dists, indices = torch.sort(scores, dim=-1, descending=True)
closest_indices = torch.gather(eval_nn_inds, -1, indices)
ranks = deepcopy(eval_nn_inds)
ranks[:, :top_k] = deepcopy(closest_indices)
ranks = ranks.cpu().data.numpy()
ranks = remove_padded_indices(ranks, eval_nn_inds, sizes)

{'map': 26.6, 'mrr': 33.46, 'precision': 13.83, 'hit_rate': 85.0, 'recall': 84.58, 'map@1': 7.39, 'mrr@1': 20.83, 'precision@1': 20.83, 'hit_rate@1': 20.83, 'recall@1': 7.39, 'map@5': 13.61, 'mrr@5': 30.81, 'precision@5': 15.67, 'hit_rate@5': 47.5, 'recall@5': 20.78, 'map@10': 16.81, 'mrr@10': 32.14, 'precision@10': 14.08, 'hit_rate@10': 57.5, 'recall@10': 29.67}


In [92]:
out = compute_metrics('viquae', ranks.T, gnd['gnd'], sizes, kappas=ks, set_name=loaders.set_name, max_sequence_len=loaders.query.dataset.max_sequence_len)

{'map': 26.6, 'mrr': 33.46, 'precision': 13.83, 'hit_rate': 85.0, 'recall': 84.58, 'map@1': 7.39, 'mrr@1': 20.83, 'precision@1': 20.83, 'hit_rate@1': 20.83, 'recall@1': 7.39, 'map@5': 13.61, 'mrr@5': 30.81, 'precision@5': 15.67, 'hit_rate@5': 47.5, 'recall@5': 20.78, 'map@10': 16.81, 'mrr@10': 32.14, 'precision@10': 14.08, 'hit_rate@10': 57.5, 'recall@10': 29.67}


In [98]:
loaders.query.dataset.max_sequence_len

500